## Make sure that you are in the right kernel (or virtual environment) before running the following code.



In [1]:
#Install the requiremnets 

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import os
import sys
from PIL import Image

# Upload the path of the folder 

In [2]:
# Write the path to the data file/ folder here:


# Add the directory of the folder you want to analyse. 
os.chdir(r'C:\Users\S.H.Y LAB\Desktop\heatmap_2024.03 (顏色區分較大)')

datafd = os.getcwd() 


In [3]:
def custom_normalize(row):
    mean_value = row.mean()
    std_dev = row.std()
    normalized_row = (row - mean_value) / std_dev
    return normalized_row

def metric_type(new_metric ,data, index, column, normalize):
    for i in range(new_metric.shape[0]):  # 遍歷每一列
        if (normalize == True):
            normalized_row = custom_normalize(data[i, :].copy())  # 創建副本，避免修改原始數據
            col_min = normalized_row.min()
            col_max = normalized_row.max()
            print(col_min)
            print(col_max)
            new_metric[i, :] = (normalized_row - col_min) / (col_max - col_min) * 100
        else :
            col_min = data[i, :].min()
            col_max = data[i, :].max()
            print(col_min)
            print(col_max)
            new_metric[i, :] = (data[i, :] - col_min) / (col_max - col_min) * 100
    metric_df = pd.DataFrame(new_metric, index, column)
    print('===============================')
    print(f"metric")
    print(metric_df.head(5))
    print('===============================')
    return metric_df

In [4]:
class Sort:
    def merge(self, left, right):
        ''' 兩數列合併 '''
        output = []
        while left and right:
            if left[0] >= right[0]:
                output.append(left.pop(0))
            else:
                output.append(right.pop(0))
        if left:
            output += left
        if right:
            output += right
        return output
    def merge_sort(self, nlst):
        ''' 合併排序 '''
        if len(nlst) <= 1:
            return nlst
        mid = len(nlst) // 2
        left = nlst[:mid]
        right = nlst[mid:]
        left = self.merge_sort(left)
        right = self.merge_sort(right)
        return self.merge(left, right)

In [5]:
'''------ Merge Sort ------'''
mergeSort = Sort()
def Sorting(new_metric ,data, index, column, normalize):
    for i in range(new_metric.shape[0]):    # i = number of cells
        if (normalize == True):
            normalized_row = custom_normalize(data[i, :].copy())  # 創建副本，避免修改原始數據 
            print(type(normalized_row)) 
            print(f"normalize shape: {normalized_row.shape}")
             
            sorted_array = mergeSort.merge_sort(list(normalized_row))
            print(f"Sorted array: {sorted_array}")
            sorted_array_length = len(sorted_array)
            
            for j in range(sorted_array_length):    # j= number of patients
                element_position = sorted_array.index(normalized_row[j])
                new_metric[i, j] = ((sorted_array_length - element_position )/ sorted_array_length) *100    # Calculate PR
                
    print(f"new metric: { new_metric }")                    
    metric_df = pd.DataFrame(new_metric, index, column)
    
    return metric_df


In [6]:

datalist = [f for f in os.listdir(datafd) if f.endswith('.xlsx')]

'''Read Excel'''
density = []
cell_type = []
info = []
patients_number = []

for i in range(len(datalist)):
    data = pd.read_excel(os.path.join(datafd, datalist[i]),engine='openpyxl')
    info.append(data.iloc[1:, :].values)



In [7]:
datalist

['DM01 quantification.T.xlsx',
 'DM02 quantification.T.xlsx',
 'DM03 quantification.T.xlsx',
 'DM04 quantification.T.xlsx',
 'DM06 quantification.T.xlsx',
 'DM08 quantification.T.xlsx',
 'DM09 quantification.T.xlsx',
 'DM10 quantification.T.xlsx',
 'DM11 quantification.T.xlsx',
 'DM12 quantification.T.xlsx',
 'DM13 quantification.T.xlsx',
 'DM14 quantification.T.xlsx',
 'DM15 quantification.T.xlsx',
 'DM16 quantification.T.xlsx',
 'DM17 quantification.T.xlsx',
 'DM18 quantification.T.xlsx',
 'DM19 quantification.T.xlsx',
 'DM20 quantification.T.xlsx',
 'DM21 quantification.T.xlsx',
 'DM22 quantification.T.xlsx',
 'DM23 quantification.T.xlsx',
 'DM24 quantification.T.xlsx',
 'DM25 quantification.T.xlsx',
 'DM26 quantification.T.xlsx',
 'DM27 quantification.T.xlsx',
 'DM28 quantification.T.xlsx',
 'DM29 quantification.T.xlsx',
 'DM30 quantification.T.xlsx',
 'DM31 quantification.T.xlsx',
 'DM32 quantification.T.xlsx',
 'DM33 quantification.T.xlsx',
 'DM35 quantification.T.xlsx',
 'DM36 q

In [8]:
'''List to Numpy'''
info = np.concatenate(info, axis=0)
print('===============================')
print("Info")
print(info)
print(info.shape)
print('===============================')



Info
[['Cytotoxic T cell' 'DM01' 3733 25.87002399517861 144.2982813118271]
 [' Activated CD8 T Cell ' 'DM01' 53 25.87002399517861 2.048703163548577]
 [' Regulatory CD8 T Cell ' 'DM01' 1474 25.87002399517861
  56.97714081265286]
 ...
 ['M2 Macrophage' 'DM38' 4322 21.68550828402367 199.3036060484753]
 ['PD-L1+M2 Macrophage' 'DM38' 92 21.68550828402367 4.242464543373375]
 ['PD-L1+ Tumor Cell' 'DM38' 704 21.68550828402367 32.46407650581365]]
(699, 5)


In [9]:
''''---Unique value and index---'''
cname1, u1a, u2a = np.unique(info[:, 0], return_index=True, return_inverse=True) # cname1 會是所有免疫細胞的名稱 u1a會是他們分別對應到的索引
cname2, u1b, u2b = np.unique(info[:, 1], return_index=True, return_inverse=True) # cname2 會是所有病人的編號，u1b會是他們分別對應的索引
print('===============================')
print("cname1 & canme2")
print(cname1)
print(cname2)
print('===============================')
print('===============================')
print("u1a & u1b")
print(u1a)
print(u1b)
print('===============================')


cname1 & canme2
[' Activated CD8 T Cell ' ' Activated NK Cell' ' CD4 CTL' ' CD8 NKT Cell'
 ' GrB+ CD8 Treg Cell ' ' GrB+ Regulatory T Cell' ' Helper T cell'
 ' NK Cell' ' Regulatory CD8 T Cell ' ' Regulatory T Cell'
 'Activated CD8 NKT Cell' 'Cytotoxic T cell' 'M1 Macrophage'
 'M2 Macrophage' 'Macrophage' 'PD-1+ cytotoxic T cell'
 'PD-1- cytotoxic T cell' 'PD-L1+ Tumor Cell' 'PD-L1+M1 Macrophage'
 'PD-L1+M2 Macrophage']
['DM01' 'DM02' 'DM03' 'DM04' 'DM06' 'DM08' 'DM09' 'DM10' 'DM11' 'DM12'
 'DM13' 'DM14' 'DM15' 'DM16' 'DM17' 'DM18' 'DM19' 'DM20' 'DM21' 'DM22'
 'DM23' 'DM24' 'DM25' 'DM26' 'DM27' 'DM28' 'DM29' 'DM30' 'DM31' 'DM32'
 'DM33' 'DM35' 'DM36' 'DM37' 'DM38']
u1a & u1b
[ 1  9  5 10  3  7  4  8  2  6 11  0 15 17 14 12 13 19 16 18]
[  0  20  40  60  80 100 120 140 160 180 200 220 240 260 280 300 320 340
 360 380 399 419 439 459 479 499 519 539 559 579 599 619 639 659 679]


In [10]:
metric = np.zeros((len(cname1), len(cname2)))
print('===============================')
print("metric")
print(metric)
print(metric.shape)
print('===============================')

metric
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0

In [11]:
'''---Map the value---'''
indtmp = np.ravel_multi_index((u2a, u2b), metric.shape)
print('===============================')
print("indtmp")
print(indtmp)
print(indtmp.shape)
print("===============================")


indtmp
[385   0 280 140 210  70 315 175 245  35 105 350 525 560 490 420 630 455
 665 595 386   1 281 141 211  71 316 176 246  36 106 351 526 561 491 421
 631 456 666 596 387   2 282 142 212  72 317 177 247  37 107 352 527 562
 492 422 632 457 667 597 388   3 283 143 213  73 318 178 248  38 108 353
 528 563 493 423 633 458 668 598 389   4 284 144 214  74 319 179 249  39
 109 354 529 564 494 424 634 459 669 599 390   5 285 145 215  75 320 180
 250  40 110 355 530 565 495 425 635 460 670 600 391   6 286 146 216  76
 321 181 251  41 111 356 531 566 496 426 636 461 671 601 392   7 287 147
 217  77 322 182 252  42 112 357 532 567 497 427 637 462 672 602 393   8
 288 148 218  78 323 183 253  43 113 358 533 568 498 428 638 463 673 603
 394   9 289 149 219  79 324 184 254  44 114 359 534 569 499 429 639 464
 674 604 395  10 290 150 220  80 325 185 255  45 115 360 535 570 500 430
 640 465 675 605 396  11 291 151 221  81 326 186 256  46 116 361 536 571
 501 431 641 466 676 606 397  12 292 152 222

In [12]:
#今天需要更動的部分是[:, 8]，8這個數字，取決於你今天要抓的數據在第幾個column
#打開excel，從左數到右，0開始數，找到要的column在第幾個位置後，再改寫[:, ？]，填入？的位置
tmpk = info[:, 4]
print('===============================')
print("tmpk")
print(tmpk)
print(tmpk.shape)
print('===============================')
metric.flat[indtmp] = tmpk
metric_df = pd.DataFrame(metric,index=cname1, columns=cname2)
print('===============================')
print("metric")
print(metric_df)
print(f"metric_df.columns[7]: {metric_df.columns[7]}")
print(f"metric_df.shape[1]: {metric_df.shape[1]}")
print('===============================')

tmpk
[144.2982813118271 2.048703163548577 56.97714081265286 20.98954373220523
 107.3443148146113 0.3865477667072786 24.23654497254637 1.005024193438924
 0.8890598634267407 0 0.6571312034023736 0.4252025433780064
 47.08507337965804 42.63383920933477 494.5362000239886 69.8721253708542
 5.22713737432457 424.6640746531344 68.60617803800999 71.58727982180446
 168.8857329467615 110.8207837304527 3.241354239216468 2.961927149628842
 8.10338559804117 0.2794270895876266 2.514843806288639 0.05588541791752531
 2.850156313793791 1.117708358350506 1.285364612103082 1.285364612103082
 12.71833268943143 54.22478329075155 325.6924384658453 115.2384063278888
 18.73376031281116 210.4540321379565 50.8733307577257 325.0908957035073
 72.4255753416435 22.08386395663228 12.3875437606199 10.52743335567058
 13.29781055453126 0.3166145370126491 2.453762661848031 0.1187304513797434
 1.543495867936665 0.831113159658204 0.1583072685063246 0.1583072685063246
 13.01429350557287 11.67537442063739 716.1074833868922 30

In [13]:
'''---metric_origin---'''
metric_percentage = np.zeros(metric.shape)
metric_percentage_df = metric_type(metric_percentage, metric, cname1, cname2, 0)

'''---metric_normalize---'''
metric_normalize_percentage = np.zeros(metric.shape)
metric_normalize_percentage_df = metric_type(metric_normalize_percentage, metric, cname1, cname2, 1)

'''---metric_log_percentage---'''
metric_log = np.log(metric+1)
metric_log_percentage = np.zeros(metric.shape)
metric_log_percentage_df = metric_type(metric_log_percentage,metric_log, cname1, cname2, 0)

'''---metric_log_normalize_percentage---'''
metric_log_percentage_normalize = np.zeros(metric_log.shape)
metric_log_percentage_normalize_df = metric_type(metric_log_percentage_normalize, metric_log, cname1, cname2, 1)



0.0
709.8203658847518
0.0
13.89468947657058
0.0
74.6634144716534
0.0
55.13765665305784
0.6567262520113974
571.005572299067
0.0
29.02446246216965
2.193069202305872
922.2640350687172
0.0385811907485991
108.683145197707
0.6567262520113974
892.0831745211534
0.6335533251105853
207.7145801433995
0.0
31.31818897893685
0.0
1315.226930026462
3.953906666695556
585.6544296727495
78.93179344476768
1150.878689861215
86.69665515921751
1736.533119533965
0.1877462429607634
1687.05164893969
3.922161934743991
781.3458735061306
0.0
3173.908402038142
0.0
114.6550717899705
0.0
465.2823144134704
metric
                            DM01       DM02      DM03       DM04      DM06  \
 Activated CD8 T Cell   0.288623  15.612511  3.111191  13.058046  1.076622   
 Activated NK Cell      0.000000   8.044141  5.981517   9.616477  1.388889   
 CD4 CTL                0.517720   0.374249  0.424056   7.911934  1.395733   
 CD8 NKT Cell           1.191801   2.331192  0.287113   2.933532  0.700000   
 GrB+ CD8 Treg Cell   

In [14]:
def plot_clustermap(metric_df, save_dir):
    
    '''Plot Setting'''
    plt.style.use("classic")
    plt.rcParams['font.family'] = 'Arial'
    plt.rcParams['axes.unicode_minus'] = False
    plt.rcParams['font.size'] = 16
    cmap = ListedColormap(sns.color_palette("RdBu_r",10))
    clustermap = sns.clustermap(metric_df, row_cluster=True, col_cluster=True, 
                                method='average', cmap=cmap, figsize=(20, 12),
                                metric='euclidean',linewidths=0.5, linecolor='black')
    clustermap.ax_heatmap.set_xticklabels(clustermap.ax_heatmap.get_xticklabels(), rotation=90)
    clustermap.ax_heatmap.set_yticklabels(clustermap.ax_heatmap.get_yticklabels(), rotation=0)
    plt.savefig(save_dir)

In [ ]:
'''Choose graph type'''

metric = [metric_percentage_df ,metric_normalize_percentage_df,metric_log_percentage_df,metric_log_percentage_normalize_df]


# What type of information do you want in your plot? Choose 0, 1, or 2 depending on the answer
#  Original metrics, Normalized percentages, Logarithm of percentages or log percentage'
ch1 = int(3) # - metric_log_percentage_df] is the default


#"What name do you want to give your plot? Do NOT write the file type (e.g., png, jpeg, etc)"
name = str('heatmap tumor density')


## HERE !! Change the directory to the folder where you want to save the plot
os.chdir(r'C:\Users\S.H.Y LAB\Desktop\heatmap_2024.03 (顏色區分較大)')
##!!!!!

current_directory = os.getcwd()
save_dir_selection = os.path.join(current_directory, f'{name}.png')

# Assuming plot_clustermap is a function that creates and saves the plot
plot_clustermap(metric[ch1],save_dir_selection)

# Open and display the saved image
img = Image.open(save_dir_selection)
img.show()
